In [27]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor # 랜덤포레스트 회귀모델
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder

#### 아파트 데이터, 금리 데이터, 학교 데이터, 지하철 주소 데이터를 통합하며 데이터 전처리와 pandas를 확실히 익힌다.

1. 데이터를 가져오고 형태를 파악한다.

In [28]:
# col_map = {
#     "RCPT_YR": "접수연도",
#     "CGG_CD": "자치구코드", > 제외
#     "CGG_NM": "자치구명",
#     "STDG_CD": "법정동코드", > 제외
#     "STDG_NM": "법정동명",
#     "LOTNO_SE": "지번구분", > 제외
#     "LOTNO_SE_NM": "지번구분명", > 제외
#     "MNO": "본번",
#     "SNO": "부번",
#     "BLDG_NM": "건물명",
#     "CTRT_DAY": "계약일", > 
#     "RTRCN_DAY": "계약해제일",
#     "BLDG_USG": "건물용도",
#     "THING_AMT": "거래금액(만원)", > 억원대로 변경
#     "ARCH_AREA": "건축면적(㎡)",
#     "LAND_AREA": "토지면적(㎡)",
#     "FLR": "해당층",
#     "ARCH_YR": "건축년도",
#     "ARCH_DECADE": "건축연대",
#     "PYEONG": "평수",
#     "PYEONG_GROUP": "평수대",
#     "PRICE_EUK": "거래금액(억원)",
#     "PRICE_PER_PYEONG": "평당가(만원/평)"
# }

In [29]:
# 데이터 불러오기

apart_df = pd.read_csv("../data/20250605_102903_seoul_real_estate.csv")
interest_rate_df = pd.read_csv("../data/deposit_lending_rate.csv")
shcool_df = pd.read_csv("../data/seoul_shcool_info.csv")
subway_df = pd.read_csv("../data/seoul_subway_encoding.csv")

In [30]:
# 지하철 컬럼명 변경 및 불필요 컬럼 삭제
subway_df.drop(columns=['Unnamed: 0', '역번호', '연번', '역전화번호', '도로명주소'], axis=1, inplace=True) # 컬럼명 없는 컬럼 삭제
subway_df.rename(columns={'호선' : 'line_num', '역명' : 'station_name', '지번주소' : 'address'}, inplace=True)

In [31]:
# 지하철 주소 컬럼 시.구분리, 주소컬럼 제거
subway_df['gu'] = subway_df['address'].str.extract(r'(\S+구)')
subway_df['dong'] = subway_df['address'].str.extract(r'(\S+(?:동|가|로))')
subway_df.drop(columns=['address','dong'], inplace=True)
subway_df['station_name'] = subway_df['station_name'] + '역'


In [32]:
# 학교 주소 컬럼 시,구분리, 주소컬럼 제거
shcool_df['gu'] = shcool_df['address'].str.extract(r'(\S+구)')
shcool_df.drop(columns=['address'], inplace=True)
shcool_df.columns

Index(['school_type', 'type', 'school_name', 'high_school_type', 'gu'], dtype='object')

In [33]:
# 금리데이터 이름 변경
interest_rate_df.rename(columns={'Unnamed: 0':'date'}, inplace=True)

In [34]:
# 금리 데이터 날짜 분리
interest_rate_df['year'] = interest_rate_df['date'].str.split('-').str[0].astype(int)
interest_rate_df['month'] = interest_rate_df['date'].str.split('-').str[1].astype(int)
interest_rate_df

,date,landing,deposit,diff,Unnamed: 4,year,month
0,1996-01,11.43,10.42,1.01,NaN,1996,1
1,1996-02,11.47,10.26,1.21,NaN,1996,2
2,1996-03,11.37,10.33,1.04,NaN,1996,3
3,1996-04,11.11,9.88,1.23,NaN,1996,4
4,1996-05,11.24,9.62,1.62,NaN,1996,5
...,...,...,...,...,...,...,...
349,2025-02,4.46,2.97,1.49,NaN,2025,2
350,2025-03,4.36,2.84,1.52,NaN,2025,3
351,2025-04,4.19,2.71,1.48,NaN,2025,4
352,2025-05,4.19,2.96,1.23,NaN,2025,5


In [35]:
# 데이터 셋 날짜 분리

# CTRT_DAY를 날짜 타입으로 변환
apart_df['CTRT_DAY'] = pd.to_datetime(apart_df['CTRT_DAY'])

# 연, 월, 일로 분리해서 새로운 컬럼 만들기
apart_df['YEAR'] = apart_df['CTRT_DAY'].dt.year
apart_df['MONTH'] = apart_df['CTRT_DAY'].dt.month

In [36]:
# 아파트 주소'구'를 이용해 지하철 역, 학교(초중고) 병합
subway_gu = subway_df.groupby('gu').first().reset_index()
apart_df = apart_df.merge(subway_gu[['line_num', 'station_name', 'gu']], right_on='gu', left_on='CGG_NM', how='left')

shcool_gu = shcool_df.groupby('gu').first().reset_index()
apart_df = apart_df.merge(shcool_gu[['school_type', 'gu']], right_on='gu', left_on='CGG_NM', how='left')


apart_df = apart_df.merge(
    interest_rate_df[['year', 'month', 'landing', 'deposit']],
    left_on=['YEAR', 'MONTH'],
    right_on=['year', 'month'],
    how='left'
)

In [37]:
# 중복 컬럼 제거 (지하철 gu, 학교 gu)
new_df = apart_df.copy()

new_df.drop(columns=['gu_x', 'gu_y'], inplace=True)

In [38]:
# 불필요 컬럼 제거 ()
# col_map = {
#     "RCPT_YR": "접수연도",
#     "CGG_CD": "자치구코드", > 제외
#     "CGG_NM": "자치구명",
#     "STDG_CD": "법정동코드", > 제외
#     "STDG_NM": "법정동명",
#     "LOTNO_SE": "지번구분", > 제외
#     "LOTNO_SE_NM": "지번구분명", > 제외
#     "MNO": "본번",
#     "SNO": "부번",
#     "BLDG_NM": "건물명",
#     "CTRT_DAY": "계약일", > 
#     "RTRCN_DAY": "계약해제일",
#     "BLDG_USG": "건물용도",
#     "THING_AMT": "거래금액(만원)", > 억원대로 변경
#     "ARCH_AREA": "건축면적(㎡)",
#     "LAND_AREA": "토지면적(㎡)",
#     "FLR": "해당층",
#     "ARCH_YR": "건축년도",
#     "ARCH_DECADE": "건축연대",
#     "PYEONG": "평수",
#     "PYEONG_GROUP": "평수대",
#     "PRICE_EUK": "거래금액(억원)",
#     "PRICE_PER_PYEONG": "평당가(만원/평)"
# }

new_df.drop(columns=["CGG_CD", "STDG_CD", "LOTNO_SE", "LOTNO_SE_NM", 'OPBIZ_RESTAGNT_SGG_NM', 'LAND_AREA'], inplace=True)

In [39]:
# 아파트 데이터 문자열 > 숫자로

# 숫자로 바꿔야 할 컬럼 리스트
str_columns = [
    'CGG_NM', 'STDG_NM', 'BLDG_NM',
    'BLDG_USG', 'DCLR_SE',
    'ARCH_DECADE', 'PYEONG_GROUP', 'BLDG_USG', 'station_name', 'school_type'
]

# LabelEncoder로 문자열을 숫자로 변환
for col in str_columns:
    le = LabelEncoder()
    new_df[col] = le.fit_transform(new_df[col].astype(str))

new_df

,RCPT_YR,CGG_NM,STDG_NM,MNO,SNO,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,FLR,...,PRICE_PER_PYEONG,YEAR,MONTH,line_num,station_name,school_type,year,month,landing,deposit
0,2025,11,253,365.0,4.0,3093,2025-06-02,38800,51.07,7,...,2511.542252,2025,6,1.0,17,5,2025,6,4.00,2.97
1,2025,1,191,447.0,0.0,5714,2025-06-02,520000,84.98,9,...,20228.386261,2025,6,2.0,13,5,2025,6,4.00,2.97
2,2025,1,8,1282.0,0.0,441,2025-06-02,320000,84.90,28,...,12459.967487,2025,6,2.0,13,5,2025,6,4.00,2.97
3,2025,16,23,1500.0,0.0,838,2025-06-02,137500,84.45,12,...,5382.421013,2025,6,2.0,14,5,2025,6,4.00,2.97
4,2025,4,315,1165.0,0.0,412,2025-06-02,97000,84.99,13,...,3772.928074,2025,6,5.0,9,4,2025,6,4.00,2.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136667,2022,7,7,491.0,0.0,215,2020-04-28,37000,83.02,5,...,1473.308234,2020,4,2.0,4,1,2020,4,2.80,1.20
136668,2022,15,145,1315.0,0.0,4855,2018-07-21,155000,101.32,14,...,5057.211747,2018,7,2.0,3,4,2018,7,3.67,1.82
136669,2022,18,9,234.0,0.0,263,2018-01-24,69905,59.96,32,...,3854.084541,2018,1,2.0,19,4,2018,1,3.69,1.80
136670,2022,15,229,70.0,0.0,5119,2013-01-23,42300,52.47,6,...,2665.041181,2013,1,2.0,3,4,2013,1,5.00,3.00


In [ ]:
# 금리데이터(시중은행 평균 금리), 서울지하철역, 서울학교 병합 데이터
new_df

,RCPT_YR,CGG_NM,STDG_NM,MNO,SNO,BLDG_NM,CTRT_DAY,THING_AMT,ARCH_AREA,FLR,...,PRICE_PER_PYEONG,YEAR,MONTH,line_num,station_name,school_type,year,month,landing,deposit
0,2025,11,253,365.0,4.0,3093,2025-06-02,38800,51.07,7,...,2511.542252,2025,6,1.0,17,5,2025,6,4.00,2.97
1,2025,1,191,447.0,0.0,5714,2025-06-02,520000,84.98,9,...,20228.386261,2025,6,2.0,13,5,2025,6,4.00,2.97
2,2025,1,8,1282.0,0.0,441,2025-06-02,320000,84.90,28,...,12459.967487,2025,6,2.0,13,5,2025,6,4.00,2.97
3,2025,16,23,1500.0,0.0,838,2025-06-02,137500,84.45,12,...,5382.421013,2025,6,2.0,14,5,2025,6,4.00,2.97
4,2025,4,315,1165.0,0.0,412,2025-06-02,97000,84.99,13,...,3772.928074,2025,6,5.0,9,4,2025,6,4.00,2.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136667,2022,7,7,491.0,0.0,215,2020-04-28,37000,83.02,5,...,1473.308234,2020,4,2.0,4,1,2020,4,2.80,1.20
136668,2022,15,145,1315.0,0.0,4855,2018-07-21,155000,101.32,14,...,5057.211747,2018,7,2.0,3,4,2018,7,3.67,1.82
136669,2022,18,9,234.0,0.0,263,2018-01-24,69905,59.96,32,...,3854.084541,2018,1,2.0,19,4,2018,1,3.69,1.80
136670,2022,15,229,70.0,0.0,5119,2013-01-23,42300,52.47,6,...,2665.041181,2013,1,2.0,3,4,2013,1,5.00,3.00
